In [2]:
import pandas as pd
import numpy as np
from sklearn import svm
from sklearn import preprocessing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV, cross_val_score

corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
corpus = corpus[corpus['qual_a_melhor_classificao_para_esse_texto:confidence'] == 1]

In [3]:
# fix labels to binary
def classFit(x):
    if x['qual_a_melhor_classificao_para_esse_texto'] == "diario":
        return 1
    else:
        return -1
    
corpus['class'] = corpus.apply(classFit,axis=1)
target = corpus['class'].values

print(corpus['qual_a_melhor_classificao_para_esse_texto'].values[:2])
print(corpus['class'][:2])

['diario' 'outro']
466    1
467   -1
Name: class, dtype: int64


In [3]:
data = TfidfVectorizer(max_features=1600).fit_transform(corpus.content)
data.shape

(534, 1600)

In [4]:
from sklearn.naive_bayes import MultinomialNB, GaussianNB

model = MultinomialNB()

precision = cross_val_score(model, data.toarray(), target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data.toarray(), target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data.toarray(), target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.700061650356
0.717327850347
0.988487394958


In [5]:
model = svm.LinearSVC(C=1)

precision = cross_val_score(model, data.toarray(), target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data.toarray(), target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data.toarray(), target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.795771464385
0.771561038542
0.872857142857


## Grid Search

In [6]:
c_range = np.logspace(-3,7,7)
param_grid = [
    {'kernel': ['rbf', 'linear'], 'C': c_range},
]
grid_search = GridSearchCV(svm.SVC(), param_grid, cv=10, verbose=3, n_jobs=10)
grid_search.fit(data, target)

Fitting 10 folds for each of 14 candidates, totalling 140 fits
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] C=0.001, kernel=rbf .............................................
[CV] .............. C=0.001, kernel=rbf, score=0.648148, total=   0.9s
[CV] .............. C=0.001, kernel=rbf, score=0.648148, total=   0.9s
[CV] C=0.001, kernel=linear ..........................................
[CV] .........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    3.5s


[CV] C=0.0464158883361, kernel=rbf ...................................
[CV] C=0.0464158883361, kernel=rbf ...................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   1.1s
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   1.1s
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   1.1s
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   1.0s
[CV] C=0.0464158883361, kernel=linear ................................
[CV] C=0.0464158883361, kernel=linear ................................
[CV] C=0.0464158883361, kernel=linear ................................
[CV] C=0.0464158883361, kernel=linear ................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   1.0s
[CV] .... C=0.0464158883361, kernel=rbf, score=0.641509, total=   1.0s
[CV] C=0.0464158883361, kernel=linear ................................
[CV] .... C=0.0464158883361, kernel=rbf, score=0.648148, total=   1.0s
[CV] .

[CV] C=4641.58883361, kernel=rbf .....................................
[CV] C=4641.58883361, kernel=rbf .....................................
[CV] ........... C=100.0, kernel=linear, score=0.711538, total=   1.2s
[CV] C=4641.58883361, kernel=rbf .....................................
[CV] ........... C=100.0, kernel=linear, score=0.788462, total=   1.1s
[CV] C=4641.58883361, kernel=rbf .....................................
[CV] ...... C=4641.58883361, kernel=rbf, score=0.722222, total=   1.1s
[CV] C=4641.58883361, kernel=linear ..................................
[CV] ...... C=4641.58883361, kernel=rbf, score=0.722222, total=   1.1s
[CV] C=4641.58883361, kernel=linear ..................................
[CV] ...... C=4641.58883361, kernel=rbf, score=0.703704, total=   1.1s
[CV] C=4641.58883361, kernel=linear ..................................
[CV] ...... C=4641.58883361, kernel=rbf, score=0.759259, total=   1.0s
[CV] C=4641.58883361, kernel=linear ..................................
[CV] .

[Parallel(n_jobs=10)]: Done 108 tasks      | elapsed:   21.8s


[CV] ... C=215443.469003, kernel=linear, score=0.703704, total=   1.0s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.703704, total=   1.0s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.703704, total=   1.0s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.759259, total=   1.0s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.777778, total=   1.0s
[CV] ... C=215443.469003, kernel=linear, score=0.796296, total=   1.0s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] ... C=215443.469003, kernel=linear, score=0.773585, total=   1.0s
[CV] C=10000000.0, kernel=rbf ........................................
[CV] C=10000000.0, kernel=rbf ........................................
[CV] .

[Parallel(n_jobs=10)]: Done 140 out of 140 | elapsed:   26.7s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params={}, iid=True, n_jobs=10,
       param_grid=[{'kernel': ['rbf', 'linear'], 'C': array([  1.00000e-03,   4.64159e-02,   2.15443e+00,   1.00000e+02,
         4.64159e+03,   2.15443e+05,   1.00000e+07])}],
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [7]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

SVC(C=2.1544346900318843, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='linear',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.769662921348
{'C': 2.1544346900318843, 'kernel': 'linear'}


In [8]:
model = svm.SVC(kernel='linear',C=2.15,gamma=0.1)

precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(precision)
print(acc)
print(recall)

0.799845479861
0.769531795947
0.861092436975


In [9]:
nb_params = { 'alpha': np.logspace(-3, 3, 7) }

grid_search = GridSearchCV(MultinomialNB(), nb_params, cv=10, verbose=3, n_jobs=10)
grid_search.fit(data, target)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.740741, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.777778, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.833333, total=   0.0s
[CV] ...........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    0.2s


[CV] ........................ alpha=1.0, score=0.685185, total=   0.0s
[CV] alpha=1000.0 ....................................................
[CV] alpha=100.0 .....................................................
[CV] ........................ alpha=1.0, score=0.750000, total=   0.0s
[CV] alpha=1000.0 ....................................................
[CV] ....................... alpha=10.0, score=0.641509, total=   0.0s
[CV] ..................... alpha=1000.0, score=0.648148, total=   0.0s
[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] ...................... alpha=100.0, score=0.653846, total=   0.0s
[CV] ..................... alpha=1000.0, score=0.648148, total=   0.0s
[CV] alpha=1000.0 ....................................................
[CV] ...................... alpha=100.0, score=0.648148, total=   0.0s
[CV] alpha=1000.0 ....................................................
[CV] alpha=100.0 .....................................................
[CV] .

[Parallel(n_jobs=10)]: Done  70 out of  70 | elapsed:    0.4s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [10]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

MultinomialNB(alpha=0.10000000000000001, class_prior=None, fit_prior=True)
0.805243445693
{'alpha': 0.10000000000000001}


In [11]:
from sklearn.naive_bayes import BernoulliNB

nb_params = { 'alpha': np.logspace(-3, 3, 7) }

grid_search = GridSearchCV(BernoulliNB(), nb_params, cv=10, verbose=3, n_jobs=10)
grid_search.fit(data, target)

Fitting 10 folds for each of 7 candidates, totalling 70 fits
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.629630, total=   0.0s
[CV] alpha=0.001 .....................................................
[CV] ...................... alpha=0.001, score=0.792453, total=   0.0s
[CV] ...................... alpha=0.001, score=0.759259, total=   0.0s
[CV] ...........

[Parallel(n_jobs=10)]: Done  12 tasks      | elapsed:    0.3s


[CV] ........................ alpha=1.0, score=0.759259, total=   0.0s
[CV] ........................ alpha=1.0, score=0.788462, total=   0.0s
[CV] ....................... alpha=10.0, score=0.685185, total=   0.0s
[CV] ........................ alpha=1.0, score=0.870370, total=   0.0s
[CV] ........................ alpha=1.0, score=0.773585, total=   0.0s
[CV] ....................... alpha=10.0, score=0.648148, total=   0.0s
[CV] alpha=10.0 ......................................................
[CV] alpha=10.0 ......................................................
[CV] alpha=10.0 ......................................................
[CV] alpha=100.0 .....................................................
[CV] ........................ alpha=0.1, score=0.811321, total=   0.0s
[CV] alpha=100.0 .....................................................
[CV] alpha=100.0 .....................................................
[CV] alpha=10.0 ......................................................
[CV] .

[Parallel(n_jobs=10)]: Done  70 out of  70 | elapsed:    0.7s finished


GridSearchCV(cv=10, error_score='raise',
       estimator=BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True),
       fit_params={}, iid=True, n_jobs=10,
       param_grid={'alpha': array([  1.00000e-03,   1.00000e-02,   1.00000e-01,   1.00000e+00,
         1.00000e+01,   1.00000e+02,   1.00000e+03])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=3)

In [12]:
print(grid_search.best_estimator_)
print(grid_search.best_score_)
print(grid_search.best_params_)

BernoulliNB(alpha=0.10000000000000001, binarize=0.0, class_prior=None,
      fit_prior=True)
0.792134831461
{'alpha': 0.10000000000000001}


In [13]:
from sklearn.tree import DecisionTreeClassifier 

model = DecisionTreeClassifier()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.727515621672
0.630793420416
0.707731092437


In [14]:
from sklearn.ensemble import RandomForestClassifier

model = RandomForestClassifier()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
precision = cross_val_score(model, data, target, cv=10, scoring='precision').mean()
print(f1)
print(acc)
print(recall)
print(precision)

0.76026130709
0.700839918293
0.809411764706
0.749167382325


In [15]:
from sklearn.linear_model import SGDClassifier

model = SGDClassifier()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.804400042797
0.739868569586
0.849915966387


In [16]:
from sklearn.neighbors import KNeighborsClassifier

model = KNeighborsClassifier()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.818654486435
0.726761812611
0.951176470588


In [17]:
from sklearn.tree import ExtraTreeClassifier

model = ExtraTreeClassifier()

f1 = cross_val_score(model, data, target, cv=10, scoring='f1').mean()
acc = cross_val_score(model, data, target, cv=10, scoring='accuracy').mean()
recall = cross_val_score(model, data, target, cv=10, scoring='recall').mean()
print(f1)
print(acc)
print(recall)

0.717652275258
0.605187335376
0.691260504202


### confidence-weighted linear classifier (Dredze et al., 2008)

In [18]:
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score, precision_score, recall_score
import oll

## manual 10-fold cross-validation
kf = KFold(n_splits=2, random_state=None, shuffle=False)

methods = ["P" ,"AP" ,"PA" ,"PA1","PA2" ,"PAK","CW" ,"AL"]

for m in methods:
    model = oll.oll(m, C=1)

    accuracy = []
    precision = []
    recall = []
    
    for train_index, test_index in kf.split(data):

        X_train, X_test = data[train_index], data[test_index]
        y_train, y_test = target[train_index], target[test_index]
        model.fit(X_train, y_train)
        predicted = model.predict(X_test)

        accuracy.append(accuracy_score(y_test, predicted))
        precision.append(precision_score(y_test, predicted))
        recall.append(recall_score(y_test, predicted))

    print(m + ': acc(' + str(np.mean(accuracy)) 
          + '), prec(' + str(np.mean(precision))
          + '), rec(' + str(np.mean(recall)) + ')'
         )

P: acc(0.797752808989), prec(0.791346524331), rec(0.953902251363)
AP: acc(0.835205992509), prec(0.847448979592), rec(0.914779379788)
PA: acc(0.803370786517), prec(0.782366211305), rec(0.98223664403)
PA1: acc(0.803370786517), prec(0.782366211305), rec(0.98223664403)
PA2: acc(0.799625468165), prec(0.780286069652), rec(0.98223664403)
PAK: acc(0.837078651685), prec(0.824587317337), rec(0.988023952096)
CW: acc(0.855805243446), prec(0.842440046793), rec(0.976248620078)
AL: acc(0.833333333333), prec(0.866954003861), rec(0.869869200147)


In [4]:
import oll

model = oll.oll("CW", C=1)
model.fit(data, target)
predicted = model.predict(data)
scores = model.scores(data)